<div class="alert alert-block alert-info">
    <img src='hardrock.png' width="200" height="200"/>
    <h1><center>Hard Rock Hotel Cancún</center></h1>  
    <p><center>Set competitivo de FACC</center></p>  
</div>

<div class="alert alert-block alert-warning">
    <h3><center>Importaciones</center></h3>
</div>

In [11]:
###
# Importadción de librerias del sistema
###
import os
import time
import datetime
from datetime import date
import random
import csv
print("Librerias del sistema listas")
###
# Importación de librerias para selenium
###
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    print("Librerias para selenium listas")
except():
    print("Instalando selenium en el enviroment")
    import sys
    !{sys.executable} -m pip install -U selenium
###
# Importación librerias urllib y BautifulSoup
###
try:
    import urllib
    from bs4 import BeautifulSoup
    print("Librerias listas de urllib y BeautifulSoup listas")
except():
    print("Instalando BeautifulSoup en el enviroment")
    import sys
    !{sys.executable} -m pip install -U BeautifulSoup
###
# Extracción del precio del dolar hoy
###
try:
    from forex_python.converter import CurrencyRates
except():
    print("Instalando la libreria CurrencyrRates")
    import sys
    !{sys.executable} -m pip install -U forex-python
###
# Importación de librerias para dataframe
###
import pandas as pd
print("Librerias para pandas listas")

Librerias del sistema listas
Librerias para selenium listas
Librerias listas de urllib y BeautifulSoup listas
Librerias para pandas listas


<div class="alert alert-block alert-warning">
    <h3><center>Métodos</center></h3>
</div>

In [12]:
def _connect_hrhc():
    """Método utilizado para conectarte a la url del hotel HRHC
       Parametros: <None>
       Retorno: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado"""
    url = "https://es.hrhcancun.com/?referrer=https%3A%2F%2Fwww.google.com%2F"
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--incognito")
    driver = webdriver.Chrome("chromedriver.exe",options=chromeOptions)
    driver.get(url)
    _wait(3)
    return driver

def _button_reserv(driver):
    """Método utilizado para levantar el submenú dereserva
       Parametros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>
    """
    str_button_reserv = 'button[class="btn btn-primary rcdBookingButton"]'
    try:
        obj_button = driver.find_element_by_css_selector(str_button_reserv)
        obj_button.click()
        _wait(3)
    except:
        print("Elemento reservar no encontrado")
    return None

def _select_dates(driver,date):
    """Método utilizado para seleccionar las fechas que nos interesan, para HRH solo es necesario hacer 
       click en un día y lo demás se seleccionará atumáticamente
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    str_chin = 'input[id="checkInDate"]'
    day = ""
    month = ""
    if today.day < 10:
        day = "0"+str(today.day)
    else:
        day = str(today.day)
    if today.month < 10:
        month = "0"+str(today.month)
    else:
        month = str(today.month)
    str_date = 'div[aria-label="'+month+'-'+day+'-'+str(today.year)+'"]'
    try:
        obj_chin = driver.find_element_by_css_selector(str_chin)
        obj_chin.click()
        _wait(2)
        try:
            obj_date = driver.find_element_by_css_selector(str_date)
            obj_date.click()
            _wait(5)
        except:
            print("Elemento date no encontrado")
    except:
        print("Elemento checkIN date no encontrado")
    return None

def _button_submit(driver):
    """Método utilizado para enviar mi petición
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    str_div = '//button[text()="Check Availability"]'
    try: 
        obj_button = driver.find_element(By.XPATH, str_div)
        obj_button.submit()
    except:
        print("Elemento div no encontrado")
    return None

def _title_habitaciones(driver):
    """Método utilizado para devolver los títulos de las habitaciones
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <lista> tipo: list --> Lista de los nombres de los hoteles"""
    list_names = []
    str_nombres = 'h4[class="kdo--rates--room-type__title"]'
    _wait(4)
    try:
        objs_names = driver.find_elements_by_css_selector(str_nombres)
        for element in objs_names:
            list_names.append(element.text)
    except:
        print("Elementos h4 no encontrados")
    return list_names

def _tarifa_room(driver):
    """Método utilizado para devolver las tarifas de las habitaciones
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <lista> tipo: list --> Lista de los precios de las habitaciones"""
    list_tarifas = []
    str_div = 'div[class="kdo-color-text--dark-gray kdo-product-box__rates-content"]'
    str_p = 'p[class="kdo-product--main-price"]'
    try:
        objs_divs = driver.find_elements_by_css_selector(str_div)
        for element in objs_divs:
            try:
                _ = element.find_element_by_css_selector(str_p)
                list_tarifas.append(_.text)
            except:
                print("Precio no encontrado, valor por defecto, 0")
                list_tarifas.append('0')
    except:
        print("Elementos divs no encontrados")
    return list_tarifas

def _tab_manage(driver,num):
    """Método utilizado para manipular las tabs del chrome driver (movernos en ellas)
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
                   <num>                tipo: int                --> Seleccionador de tab
       Retorno: <None>"""
    try:
        window_after = driver.window_handles[num]
        driver.switch_to_window(window_after)
        time.sleep(3)
    except:
        print("Elemento tab no encontrada")
    return None

def _tab_closer(driver):
    """Método utilizado para cerrar los tabs
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    try:
        driver.close()
    except:
        print("Elemento tab cerrado anteriormente")
    return None

def _validator_hab(lista2):
    """Método utilizado para limpiar la lista del precio de las habitaciones
       Parámetros: <lista2> tipo: list --> Lista2 es el precio de las habitaciones
       Retorno: <lista2> tipo: list --> Lista2 ya limpia"""
    return list(map(lambda x:x.replace('MXN$','').replace(',','') if x.find('MXN')== 0 else x,lista2))

def _fill_df(df,date_consult,dateIN,dateOUT,nombres,precios,dolar):
    """Método utilizado para llenar el dataframe
       Parámetros: <df>           tipo: Pandas.DataFrame --> El dataframe instanciado 
                   <date_consult> tipo: Date             --> La fecha en que se realizó esta extracción
                   <dateIN>       tipo: str              --> La fecha de llegada
                   <dateOUT>      tipo: str              --> La fecha de salida
                   <nombres>      tipo: list             --> Los nombres de las habitaciones
                   <precios>      tipo: list             --> Los precios de las habitaciones
                   <dolar>        tipo: float            --> El precio del dólar al día
       Retorno: <df> tipo: Pandas:DataFrame --> El dataframe lleno con estos n resultados"""
    dateOUT = dateIN + datetime.timedelta(days=1)
    #dateIN = str(dateIN.year)+'/'+str(dateIN.month)+'/'+str(dateIN.day)
    #dateOUT = str(dateOUT.year)+'/'+str(dateOUT.month)+'/'+str(dateOUT.day)
    for indx in range(len(nombres)):
        dicc_data = {"Hotel":"Hard Rock Hotel Cancún",
                 "Fecha_consulta":str(consult_Date),
                 "Fecha_reserva_checkin":str(dateIN),
                 "Fecha_reserva_checkout":str(dateOUT),
                 "Nombre_habitacion":nombres[indx],
                 "Precio_habitacion_MXN":precios[indx],
                 "Precio_habitacion_USD":float(precios[indx])/dolar,
                 "Precio_dolar_al_dia":dolar}
        df = df.append(dicc_data,ignore_index = True)
    return df

def _select_habt(df):
    """Método utilizado para seleccionar solo las habitaciones que nos interesan
       Párametros: <df> tipo: Pandas:DataFrame --> El dataframe ya lleno
       Retorno: <df> tipo: Pandas:DataFrame --> El data frame ya filtrado"""
    return df.loc[(df['Nombre_habitacion']=='Deluxe Room')|\
                  (df['Nombre_habitacion']=='Deluxe Gold - King Bed')|\
                  (df['Nombre_habitacion']=='Deluxe Gold - Double Beds')|\
                  (df['Nombre_habitacion']=='Deluxe Platinum - King Bed')|\
                  (df['Nombre_habitacion']=='Deluxe Platinum - Double Beds')|\
                  (df['Nombre_habitacion']=='Deluxe Diamond'),:]
    
def _wait(num):
    """Método utilizado para realizar una espera implicita
       Parámetros: <num> tipo: int --> Requiere de un número para indicar una espera
       Retorno: <None>"""
    time.sleep(num)

def _quit(driver):
    """Método utilizado para cerrar el driver
       Parámetros: <selenium.webdriver> tipo: selenium.webdriver --> Es el bot ya levantado y configurado
       Retorno: <None>"""
    driver.quit()
    return None

<div class="alert alert-block alert-warning">
    <h3><center>Implementación</center></h3>
</div>

In [13]:
###
# Obtención del precio del dólar
###
c = CurrencyRates()
mxn = c.get_rate('USD', 'MXN')
mxn

24.2688102894

In [14]:
num_days = int(input("Ingresa el número de días a consultar: "))
today = datetime.date.today()
consult_Date = date.today()
dicc_hrhc = {"Hotel":[],
             "Fecha_consulta":[],
             "Fecha_reserva_checkin":[],
             "Fecha_reserva_checkout":[],
             "Nombre_habitacion":[],
             "Precio_habitacion_MXN":[],
             "Precio_habitacion_USD":[], 
             "Precio_dolar_al_dia":[]}
hrhc_data = pd.DataFrame(dicc_hrhc)
driver = _connect_hrhc()
_wait(3)
while num_days > 0:
    _button_reserv(driver)
    _select_dates(driver,today)
    _button_submit(driver)
    _tab_manage(driver,1)
    listan = _title_habitaciones(driver)
    listah = _tarifa_room(driver)
    print("Antes de validar ",listah)
    listah = _validator_hab(listah)
    print("Despues de validar ",listah)
    hrhc_data = _fill_df(hrhc_data,consult_Date,today,today,listan,listah,mxn)
    hrhc_data = _select_habt(hrhc_data)
    _tab_closer(driver)
    _tab_manage(driver,0)
    num_days -= 1
    today = today + datetime.timedelta(days=1)
_quit(driver)

Ingresa el número de días a consultar: 90


C:\Users\israel.vargas\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:112: DeprecationWarning: use driver.switch_to.window instead


Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Antes de validar  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Despues de validar  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Elemento checkIN date no encontrado
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor po

Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Antes de validar  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Despues de validar  ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
Elemento checkIN date no encontrado
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor por defecto, 0
Precio no encontrado, valor po

Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,720', 'MXN$6,014', 'MXN$6,014', 'MXN$6,161', 'MXN$6,161', 'MXN$7,632', 'MXN$8,367', 'MXN$9,985', 'MXN$9,985', 'MXN$14,102']
Despues de validar  ['5720', '5720', '5720', '6014', '6014', '6161', '6161', '7632', '8367', '9985', '9985', '14102']
Elemento checkIN date no encontrado
Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,720', 'MXN$6,014', 'MXN$6,014', 'MXN$6,161', 'MXN$6,161', 'MXN$7,632', 'MXN$8,367', 'MXN$9,985', 'MXN$9,985', 'MXN$14,102']
Despues de validar  ['5720', '5720', '5720', '6014', '6014', '6161', '6161', '7632', '8367', '9985', '9985', '14102']
Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,720', 'MXN$6,014', 'MXN$6,014', 'MXN$6,161', 'MXN$6,161', 'MXN$7,632', 'MXN$8,367', 'MXN$9,985', 'MXN$9,985', 'MXN$14,102']
Despues de validar  ['5720', '5720', '5720', '6014', '6014', '6161', '6161', '7632', '8367', '9985', '9985', '14102']
Elemento checkIN date no encontrado
Antes de validar  ['MXN$5,720', 'MXN$5,720', '

Precio no encontrado, valor por defecto, 0
Antes de validar  ['MXN$8,562', 'MXN$9,011', 'MXN$9,011', 'MXN$9,249', 'MXN$9,249', 'MXN$9,412', 'MXN$9,412', 'MXN$10,562', 'MXN$11,212', 'MXN$18,011', 'MXN$28,598', '0']
Despues de validar  ['8562', '9011', '9011', '9249', '9249', '9412', '9412', '10562', '11212', '18011', '28598', '0']
Elemento checkIN date no encontrado
Precio no encontrado, valor por defecto, 0
Antes de validar  ['MXN$8,562', 'MXN$9,011', 'MXN$9,011', 'MXN$9,249', 'MXN$9,249', 'MXN$9,412', 'MXN$9,412', 'MXN$10,562', 'MXN$11,212', 'MXN$18,011', 'MXN$28,598', '0']
Despues de validar  ['8562', '9011', '9011', '9249', '9249', '9412', '9412', '10562', '11212', '18011', '28598', '0']
Precio no encontrado, valor por defecto, 0
Antes de validar  ['MXN$8,562', 'MXN$9,011', 'MXN$9,011', 'MXN$9,249', 'MXN$9,249', 'MXN$9,412', 'MXN$9,412', 'MXN$10,562', 'MXN$11,212', 'MXN$18,011', 'MXN$28,598', '0']
Despues de validar  ['8562', '9011', '9011', '9249', '9249', '9412', '9412', '10562', 

Elemento checkIN date no encontrado
Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,721', 'MXN$6,015', 'MXN$6,015', 'MXN$6,162', 'MXN$6,162', 'MXN$7,633', 'MXN$8,368', 'MXN$9,986', 'MXN$9,986', 'MXN$14,103']
Despues de validar  ['5720', '5720', '5721', '6015', '6015', '6162', '6162', '7633', '8368', '9986', '9986', '14103']
Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,721', 'MXN$6,015', 'MXN$6,015', 'MXN$6,162', 'MXN$6,162', 'MXN$7,633', 'MXN$8,368', 'MXN$9,986', 'MXN$9,986', 'MXN$14,103']
Despues de validar  ['5720', '5720', '5721', '6015', '6015', '6162', '6162', '7633', '8368', '9986', '9986', '14103']
Elemento checkIN date no encontrado
Antes de validar  ['MXN$5,720', 'MXN$5,720', 'MXN$5,721', 'MXN$6,015', 'MXN$6,015', 'MXN$6,162', 'MXN$6,162', 'MXN$7,633', 'MXN$8,368', 'MXN$9,986', 'MXN$9,986', 'MXN$14,103']
Despues de validar  ['5720', '5720', '5721', '6015', '6015', '6162', '6162', '7633', '8368', '9986', '9986', '14103']
Antes de validar  ['MXN$5,720', 'MXN$5,720', '

In [15]:
###
# Guardando los archivos
###
hrhc_data.to_csv("./HRHC_data/HRHC_"+str(consult_Date)+"_comma.csv",quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
hrhc_data.to_csv("./HRHC_data/HRHC_"+str(consult_Date)+"_pipe.csv",sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")